# SynIG: Synthetic InfoGroup

## Created columns

- ABI: fake
- COMPANY: fake
- ADDRESS: fake
- CITY: fake
- ZIP: fake
- COUNTY_CODE: sample
- STATE: sample
- FIPS_CODE: STATE + COUNTY_CODE
- LATITUDE: sample + noise
- LONGITUDE: sample + noise
- YEAR: 1997-2017
- NAICS: sample, 6 digits
- EMPLOYEES: modelled
- EMPLOYEES_CODE: derived from EMPLOYEES
- SALES: derived from EMPLOYEES + noise
- SALES_CODE: derived from SALES

## Generation algorithm

### Fake data

Generated using [`mimesis`](https://github.com/lk-geimfari/mimesis) package for US locale.

### Sample

Sampled from InfoGroup separately for each state-sector, pooled across years.

### Generated

state space for each establishment:
- size (employees)
- age (automatic)
- 2-digit NAICS (constant) 
- US state (constant)

STATE x NAICS - some cells will be small, so use clustering to attach smallest cells. [Hierarchical clustering](https://scikit-learn.org/stable/modules/clustering.html#hierarchical-clustering) maybe be a good approach. once data is generated, randomly assign each establishment from cluster to cells.

initial state t0 (year 1997): fit cross-section distribution and draw desired N

iteration for t:
- draw exit probability, conditional on state at t-1
- draw growth probability (because many do not grow)
- conditional on growth, draw arc growth rate
  - maybe fit [truncated normal](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.truncnorm.html) distribution
  - since establishments with 1-5 employees constitute 75% of data, maybe fit some descrete growth distribution for them
- compute $\Delta$EMP from agr
  - round up values < 1.
- draw sample of births
  - number equal to deaths: stable population
  - birth rate conditional on year: dynamic population
  - birth rate conditional on year and naics-state

add other (random) variables: 6-digit naics, street address, ...


In [ ]:
import itertools
import shutil

import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import scipy.stats
import mimesis
import fastparquet

from rurec import infogroup, resources
# memory = joblib.Memory('../cache')
memory = joblib.Memory(None) # disable caching

[NAICS sectors](https://www.census.gov/cgi-bin/sssd/naics/naicsrch?chart=2017)

```
Sector 	Description
11 	Agriculture, Forestry, Fishing and Hunting
21 	Mining, Quarrying, and Oil and Gas Extraction
22 	Utilities
23 	Construction
31-33 	Manufacturing
42 	Wholesale Trade
44-45 	Retail Trade
48-49 	Transportation and Warehousing
51 	Information
52 	Finance and Insurance
53 	Real Estate and Rental and Leasing
54 	Professional, Scientific, and Technical Services
55 	Management of Companies and Enterprises
56 	Administrative and Support and Waste Management and Remediation Services
61 	Educational Services
62 	Health Care and Social Assistance
71 	Arts, Entertainment, and Recreation
72 	Accommodation and Food Services
81 	Other Services (except Public Administration)
92 	Public Administration
```

In [ ]:
def naics_sector(x):
    """Return 2-digit NAICS sector from full NAICS code.
    Sectors with multiple codes are coded with just the first,
    eg. "31-33 Manufacturing" is coded as "31".
    """
    join_multiple = {'32': '31', '33': '31', '45': '44', '49': '48'}
    return x.str[:2].replace(join_multiple)

naics_sectors = ['11', '21', '22', '23', '31', '42', '44', '48', '51', '52', '53', '54', '55', '56', '61', '62', '71', '72', '81', '92']

state_fips_code = {
    'AL':'01',
    'AK':'02',
    'AS':'60',
    'AZ':'04',
    'AR':'05',
    'CA':'06',
    'CO':'08',
    'CT':'09',
    'DE':'10',
    'DC':'11',
    'FL':'12',
    'FM':'64',
    'GA':'13',
    'GU':'66',
    'HI':'15',
    'ID':'16',
    'IL':'17',
    'IN':'18',
    'IA':'19',
    'KS':'20',
    'KY':'21',
    'LA':'22',
    'ME':'23',
    'MH':'68',
    'MD':'24',
    'MA':'25',
    'MI':'26',
    'MN':'27',
    'MS':'28',
    'MO':'29',
    'MT':'30',
    'NE':'31',
    'NV':'32',
    'NH':'33',
    'NJ':'34',
    'NM':'35',
    'NY':'36',
    'NC':'37',
    'ND':'38',
    'MP':'69',
    'OH':'39',
    'OK':'40',
    'OR':'41',
    'PW':'70',
    'PA':'42',
    'PR':'72',
    'RI':'44',
    'SC':'45',
    'SD':'46',
    'TN':'47',
    'TX':'48',
    'UM':'74',
    'UT':'49',
    'VT':'50',
    'VA':'51',
    'VI':'78',
    'WA':'53',
    'WV':'54',
    'WI':'55',
    'WY':'56'
}

# State-sector distribution

Supress cells with less than 1000 establishments per year.

In [ ]:
counts = {}
for state in state_fips_code.keys():
    df = infogroup.get_df(cols=['NAICS'], states=[state])
    df['SECTOR'] = naics_sector(df['NAICS'])
    counts[state] = df.groupby('SECTOR').size()
df = pd.concat(counts, 1)

In [ ]:
n_years = 21
min_est = 1000
s = df.stack().dropna()
s = s[s > n_years * min_est]
state_sector_weights = s / s.sum()
joblib.dump(state_sector_weights, resources.paths.root / 'tmp/state_sector_weights')

In [ ]:
state_sector_weights = joblib.load(resources.paths.root / 'tmp/state_sector_weights')
state_sector_weights.index = state_sector_weights.index.swaplevel()

# Faking

In [ ]:
def gen_fake(size):
    faker = mimesis.Generic('en')
    data = []
    for _ in range(size):
        company = faker.business.company().upper()
        address = faker.address.address().upper()
        city = faker.address.city().upper()
        zip = faker.address.zip_code()
        data.append((company, address, city, zip))
    return pd.DataFrame(data, columns=['COMPANY', 'ADDRESS', 'CITY', 'ZIP'])

gen_fake(10)

# Sampling

In [ ]:
@memory.cache
def get_sampling_source(state, sector):
    df = infogroup.get_df(cols=['ABI', 'NAICS', 'COUNTY_CODE', 'LATITUDE', 'LONGITUDE'], states=[state])
    del df['YEAR']
    df = df[naics_sector(df['NAICS']) == sector]
    df = df.drop_duplicates('ABI').dropna()
    del df['ABI']
    df['NAICS'] = df['NAICS'].str[:6]
    return df.reset_index(drop=True)

def gen_sample(state, sector, size):
    src = get_sampling_source(state, sector)
    df = src[['COUNTY_CODE', 'LATITUDE', 'LONGITUDE']].sample(size, replace=True).reset_index(drop=True)
    df['NAICS'] = src['NAICS'].sample(size, replace=True).reset_index(drop=True)
    df['LATITUDE'] += sp.stats.uniform.rvs(-0.5, 1, size=size)
    df['LONGITUDE'] += sp.stats.uniform.rvs(-0.5, 1, size=size)
    df['STATE'] = state
    df['FIPS_CODE'] = df['STATE'].replace(state_fips_code) + df['COUNTY_CODE']
    return df

In [ ]:
df = gen_sample('WI', '23', 1000)
plt.scatter('LONGITUDE', 'LATITUDE', data=df)

# Modeling

In [ ]:
@memory.cache
def get_model_src(state, sector):
    
    data_years = range(1997, 2018)

    df = infogroup.get_df(cols=['ABI', 'YEAR', 'NAICS', 'EMPLOYEES'], states=[state])
    df = df.loc[naics_sector(df['NAICS']) == sector, ['YEAR', 'ABI', 'EMPLOYEES']]
                
    df = df.sort_values(['ABI', 'YEAR'])
    df['EMPLOYEES_LAG'] = df.groupby('ABI')['EMPLOYEES'].shift()
    df['YEAR_LAG'] = df.groupby('ABI')['YEAR'].shift()
    df['YEAR_LEAD'] = df.groupby('ABI')['YEAR'].shift(-1)

    df['BIRTH'] = df['YEAR_LAG'].isna()
    df.loc[df['YEAR'] == data_years[0], 'BIRTH'] = np.nan
    df['DEATH'] = df['YEAR_LEAD'].isna()
    df.loc[df['YEAR'] == data_years[-1], 'DEATH'] = np.nan

    
    return df[['EMPLOYEES', 'EMPLOYEES_LAG', 'BIRTH', 'DEATH']].reset_index(drop=True)

get_model_src('WI', '23').head(2)

In [ ]:
class Birth:
    """Generate random series of given `size` that resembles distribution of 1-d array `x`.
    """
    def fit(self, x):
        x = np.array(x)
        x = x[(0 < x) & (x < 100)]
        x = np.log10(x)
        b = np.unique(np.quantile(x, np.linspace(0, 1, 50)))
        self.rv = sp.stats.rv_histogram(np.histogram(x, b))()
    def predict(self, size=1):
        return (10**self.rv.rvs(size)).round()

d = sp.stats.lognorm.rvs(1.5, size=10_000)
d = d[d < 150]
m = Birth()
m.fit(d)
dh = m.predict(len(d))

plt.hist([d, dh])
plt.yscale('log')

In [ ]:
class Death:
    """Generate random series of death indicator, predicted from 1-d array `x`.
    Death probability for Bernoulli draws is estimated in each bin formed by `breaks`.
    """
    def __init__(self, breaks=None):
        if breaks is None:
            breaks = [0, np.inf]
        self.bins = pd.IntervalIndex.from_breaks(breaks)
    def fit(self, x, y):
        x = np.array(x)
        y = np.array(y)
        d = pd.DataFrame({'x': x, 'y': y, 'c': pd.cut(x, self.bins)})
        p = d.groupby(['c', 'y']).size().unstack()
        self.prob = (p[1] / p.sum(1)).fillna(0)
    def predict(self, x):
        x = np.array(x)
        d = pd.DataFrame({'x': x, 'c': pd.cut(x, self.bins)}).sort_values('c')
        counts = d['c'].value_counts().sort_index()
        y = []
        for b, size in counts.iteritems():
            y.append(sp.stats.bernoulli.rvs(self.prob[b], size=size))
        d['y'] = np.concatenate(y)
        return d['y'].sort_index().values
            
        

d = pd.DataFrame({'x': [1,1,1,1,1,6,7,8] * 1000,
                  'y': [0,0,1,1,1,0,0,1] * 1000})
m = Death()
m.fit(d['x'], d['y'])
assert (m.prob == [0.5]).all()
assert np.isclose(m.predict(sp.stats.randint.rvs(1, 10, size=1000)).mean(), 0.5, atol=0.03)

m = Death([0, 1, 5, 10])
m.fit(d['x'], d['y'])
assert (m.prob == [3/5, 0, 1/3]).all()
d['c'] = pd.cut(d['x'], m.bins)
d['yh'] = m.predict(d['x'])
t = d.groupby('c')[['y', 'yh']].mean()
assert np.allclose(t['y'], t['yh'], atol=0.03, equal_nan=True)

Arc growth rate: $g = \frac{x_1 - x_0}{0.5x_0 + 0.5x_1}$

$x_1 = x_0\frac{2+g}{2-g}$

In [ ]:
class Growth:
    """Generate random values for next period."""
    def fit(self, x0, x1):
        x0 = np.array(x0)
        x1 = np.array(x1)
        gr_mask = (x0 != x1)
        self.gr_prob = gr_mask.mean()
        x0 = x0[gr_mask]
        x1 = x1[gr_mask]
        gr = (x1 - x0) * 2 / (x0 + x1)
        
        a, b = -1.5, 1.5
        rv_par = sp.stats.truncnorm.fit(gr[(a < gr) & (gr < b)], fix_a=a, fix_b=b)
        self.rv = sp.stats.truncnorm(*rv_par)
        
    def predict(self, x0):
        x0 = np.array(x0)
        d = pd.DataFrame({'x0': x0})
        d['growth'] = sp.stats.bernoulli.rvs(self.gr_prob, size=len(x0)).astype('bool')
        d['x1'] = np.nan
        d.loc[~d['growth'], 'x1'] = d['x0']
        gr = self.rv.rvs(d['growth'].sum())
        d.loc[d['growth'], 'x1'] = d.loc[d['growth'], 'x0'] * (2 + gr) / (2 - gr)
        d['x1'] = np.ceil(d['x1'])
        d.loc[d['growth'] & (d['x1'] == d['x0']) & (d['x1'] > 1), 'x1'] -= 1
        return d['x1'].values

n = 10_000
gr_prob = 0.2
x0 = sp.stats.randint.rvs(1, 20, size=n)
x1 = sp.stats.randint.rvs(1, 20, size=n)
gr_mask = sp.stats.bernoulli.rvs(gr_prob, size=n) == 0
x1[gr_mask] = x0[gr_mask] 
g = Growth()
g.fit(x0, x1)
xh = g.predict(x0)
plt.hist([x1-x0, xh-x0], bins=range(-20, 21))
plt.yscale('log')

In [ ]:
@memory.cache
def gen_model_data(state, sector, size=None):
    syn_years = range(2001, 2021)

    df = get_model_src(state, sector)
    
    if size is None:
        size = len(df)
    size_per_year = size // len(syn_years)

    init = Birth()
    init.fit(df['EMPLOYEES'].dropna())

    birth = Birth()
    birth.fit(df.loc[df['BIRTH'] == 1, 'EMPLOYEES'].dropna())

    death = Death([0, 1, 5, 20, 50, np.inf])
    d = df[['EMPLOYEES', 'DEATH']].dropna()
    death.fit(d['EMPLOYEES'], d['DEATH'])

    growth = Growth()
    d = df[['EMPLOYEES', 'EMPLOYEES_LAG']].dropna()
    growth.fit(d['EMPLOYEES_LAG'], d['EMPLOYEES'])

    df = []

    next_est_id = 1
    emp0 = init.predict(size_per_year)
    df0 = pd.DataFrame({'ABI': range(next_est_id, size_per_year + 1), 'EMPLOYEES': emp0})
    next_est_id = size_per_year + 1
    df0['DEATH'] = death.predict(emp0)
    df0['YEAR'] = syn_years[0]
    df.append(df0)

    for year in syn_years[1:]:
        emp1_cont = growth.predict(df0.loc[df0['DEATH'] == 0, 'EMPLOYEES'])
        death_count = (df0['DEATH'] == 1).sum()
        emp1_bir = birth.predict(death_count)
        est_id1 = df0.loc[df0['DEATH'] == 0, 'ABI'].append(pd.Series(range(next_est_id, next_est_id + death_count)))
        next_est_id = next_est_id + death_count
        df1 = pd.DataFrame({'ABI': est_id1, 'EMPLOYEES': np.concatenate([emp1_cont, emp1_bir])})
        df1['DEATH'] = death.predict(df1['EMPLOYEES'])
        df1['YEAR'] = year
        df.append(df1)
        df0 = df1

    df = pd.concat(df, ignore_index=True)
    
    return df


gen_model_data('WI', '23').head(5)

# Add constant columns

In [ ]:
def gen_chunk(state, sector, size):
    df = gen_model_data(state, sector, size)
    ids = df['ABI'].unique()
    dfc = gen_sample(state, sector, len(ids))
    dfc = pd.concat([dfc, gen_fake(len(ids))], 1)
    dfc['ABI'] = ids
    df = df.merge(dfc, 'left', 'ABI')
    return df

In [ ]:
import logging
logging.basicConfig(filename=resources.paths.root / 'logs/synth.log',filemode='w', level=logging.INFO, format='%(asctime)s %(levelname)s: %(message)s', force=True)

In [ ]:
size_per_year = 1_000_000
n_years = 20
total_size = size_per_year * n_years

pq_path = resources.paths.root / 'data/synig.pq'
if pq_path.exists():
    shutil.rmtree(pq_path)
pq_path.mkdir()

In [ ]:
partition_paths = []
for state, sector in state_sector_weights.index:
    logging.info(f'{state}, {sector}')
    p = pq_path / f'STATE={state}/SECTOR={sector}'
    if not p.exists():
        w = state_sector_weights[(state, sector)]
        df = gen_chunk(state, sector, int(w * total_size))
        del df['STATE']
        fastparquet.write(str(p), df, file_scheme='hive', write_index=False, partition_on=['YEAR'])
    partition_paths.append(str(p))
fastparquet.writer.merge(partition_paths)

# Save as CSVs

In [ ]:
csv_path = resources.paths.root / 'data/synig'
if csv_path.exists(): shutil.rmtree(csv_path)
csv_path.mkdir()
pf = fastparquet.ParquetFile(str(pq_path))
for year in range(2001, 2021):
    df = pf.to_pandas(filters=[('YEAR', '==', year)])
    df['ABI'] = df['ABI'].astype(str).str.zfill(9)
    df['NAICS'] += '00'
    df.loc[df.sample(frac=0.06).index, 'EMPLOYEES'] = np.nan
    df['EMPLOYEES'] = df['EMPLOYEES'].astype('Int32')
    df = df[['YEAR', 'STATE', 'SECTOR', 'ABI', 'NAICS', 'EMPLOYEES', 'COMPANY', 'ADDRESS', 'CITY', 'ZIP', 'COUNTY_CODE', 'FIPS_CODE', 'LONGITUDE', 'LATITUDE']]
    df.to_csv(csv_path / f'{year}.csv', index=False)